<a href="https://colab.research.google.com/github/lsmanoel/BasicOfPython/blob/master/BasicOfTensorflow/Tensorboard_basicExample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quick guide to run TensorBoard in Google Colab

from [dlology.com](https://www.dlology.com/blog/quick-guide-to-run-tensorboard-in-google-colab/)

The Google Colab virtual machine is running on a local network located in a Google's server that could be anywhere in the world. This virtual machine is a unix system that have a firewall. This firewall blocks to access the TensorBoard page from Google Colab virtual machine server. Through the [ngrok](https://ngrok.com/) service we able to tunnel the connection to Colab server.

![alt text](https://gitcdn.xyz/cdn/Tony607/blog_statics/d425c3fe4cf0d92067572e25ae6cc3198d51936b//images/ngrok/ngrok.jpg)

**Setup ngrok and run TensorBoard on Colab:**

Ngrok executable can be directly downloaded to your Colab notebook, run those two lines of code.

In [0]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2019-01-21 13:34:50--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.200.123.104, 52.2.175.150, 52.203.102.189, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.200.123.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5363700 (5.1M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]   5.11M  --.-KB/s    in 0.1s    

2019-01-21 13:34:50 (42.1 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [5363700/5363700]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

After this, the executable file ngrok will be extracted to the current directory.

Next, let's fire up the TensorBoard in the background like this:

In [0]:
LOG_DIR = 'log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

It is assuming the TensorBoard log path is "./log", where we are going to tell Keras to log files.

Then, we can run ngrok to tunnel TensorBoard port 6006 to the outside world. This command also runs in the background.

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

One last step, we get the public URL where we can access the colab TensorBoard web page.

In [0]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

This will output a URL you can click on, but wait! We haven't trained our model, so you not get any information from TensorBoard yet.

**TensorBoard for Keras:**

In this section, we will start training a Keras model and ask Keras to output TensorBoard log files to the ./log directory.

Keras output TensorBoard log files by callbacks, which allows you to visualize dynamic graphs of your training and test metrics, as well as activation histograms for the different layers in your model. You can choose whether to visualize individual components and even how frequently you want Keras to activation and weight histograms. The following code will allow you to visualize all available components of our Keras model. The model itself is a simple two layers of 3x3 convolutional followed by 2 Dense layers to classify MNIST handwritten digits dataset.

In [0]:
'''Trains a simple convnet on the MNIST dataset.

Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.callbacks import TensorBoard

batch_size = 10000
num_classes = 10
epochs = 3

# input image dimensions
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])


tbCallBack = TensorBoard(log_dir='log', histogram_freq=1,
                         write_graph=True,
                         write_grads=True,
                         batch_size=batch_size,
                         write_images=True)

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[tbCallBack])
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

---
# A short tour to TensorBoard

**SCALARS:**

In this tab, you can see four graphs, acc, loss, acc_val, and loss_val, representing training accuracy, training loss, validation accuracy and validation loss. During an ideal training process, we are expecting the accuracies to increase, and losses decrease over time. However, if you see the validation accuracy start to going downhill after training for some specific epochs. Your model might get overfit on the training sets. I have a quick [Two Simple Recipes for Over Fitted Model](https://www.dlology.com/blog/two-simple-recipes-for-over-fitted-model/). While there are other reasons why a model gets overfit, one apparent reason is the training data is too small, and the solution is either find or generate more data.

![alt text](https://gitcdn.xyz/cdn/Tony607/blog_statics/d425c3fe4cf0d92067572e25ae6cc3198d51936b//images/ngrok/scalars.png)

Our model's losses and accuracies match our expectation.

**HISTOGRAMS:**

This tab displays the distribution of activation weights, bias, gradients histograms for different layers in your model. One way this graph can be useful is by visualizing the value of gradients we can tell if a model is suffered from vanishing/exploding gradients, give my previous post a read How to deal with Vanishing/Exploding gradients in Keras.

![alt text](https://gitcdn.xyz/cdn/Tony607/blog_statics/d425c3fe4cf0d92067572e25ae6cc3198d51936b//images/ngrok/histograms_conv2d_1.png)

For other tabs, graphs tab shows the computation graph, more useful if you are building a custom layer or loss function. Image tab visualizes the model weights as images.



**Summary and further reading:**

You learned how to run TensorBoard on a Google Colab notebook and access it on your local machine by leveraging the free ngrok tunneling service.

One bonus, alternatively you can run the following code in Colab to use localtunnel instead of ngrok.

In [0]:
LOG_DIR = './log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
# Install
! npm install -g localtunnel

# Tunnel port 6006 (TensorBoard assumed running)
get_ipython().system_raw('lt --port 6006 >> url.txt 2>&1 &')

# Get url
! cat url.txt

To develop more understanding of configuring TensorBoard with Keras, refer to the [official document](https://keras.io/callbacks/#tensorboard), you can disable some of the unwanted features to speed up the training since generating logs for TensorBoard takes a substantial amount of time.

